### Read addresses json data

In [1]:
import os
import json

json_file_path = "../addresses/addresses.json"
data = {}
if os.path.exists(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

In [ ]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
chunk_size = 8000
def ask_gpt4(addresses):
    result = {}
    for year, address in addresses.items():
        i = 0
        address_analysis = {}
        while True:
            chunk = ""
            if len(address) > i+chunk_size:
                chunk = address[i*chunk_size:i*chunk_size+chunk_size]
            else:
                chunk = address[i*chunk_size:]

            text = chunk + '''
                Imagine you're a political analyst and you need to return the result in JSON form in following format:
                {
                    "key phrase1": [score1],
                    "key phrase2": [score2],
                    "key phrase3": [score3],
                    "key phrase4": [score4],
                    "key phrase5": [score5],
                    "key phrase6": [score6],
                    "key phrase7": [score7],
                    "key phrase8": [score8],
                    "key phrase9": [score9],
                    "key phrase10": [score10],
                }
                Here the "key phrase" is a pair of words or a concept that the President uses, and the "score" is some value  that shows how the phrase important in exactly this address. Here's the scale:
                5 - Very important and affects all 3 areas
                4 - Important, but affects at least 2 areas
                3 - Important, but affects at least 1 areas
                2 - Somewhat important, and slightly affects 1 area
                1 - Not important

                IMPORTANT: return nothing but the result in JSON format, so that it can be parsed with Python.
            '''
            i += 1
            message = [{"role": "system", "content": text}]
            chat = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", messages=message
            )
            reply = chat.choices[0].message.content
            try:
                dict_reply = json.loads(reply)
            except:
                print("Problem with chunk ", i)
                print(text)
                continue

            address_analysis[i+1] = dict_reply
            if len(address) < i+chunk_size:
                break

        result[year] = address_analysis
        print(result)

        break

ask_gpt4(data)